# Computing seasonal means from CFSR data

## Introduction

These notes will show how to:
1. Access the 6-hourly CFSR dataset on our local disks
2. Use `xarray.open_mfdataset()` to concatenate data spread across many individual netCDF files into a single dataset
3. Take advantage of xarray's "lazy execution" model to define our climatology before explicitly computing any results
4. Compute the climatology (seasonal means) and save the resulting (vastly) reduced datasets to disk as netCDF files

***Note, this code takes a long time to run! And there is really no need to run it again because we are saving the results to disk.***

This is just for demonstration purposes.

## The CFSR data

The [Climate Forecase System Reanalysis](https://climatedataguide.ucar.edu/climate-data/climate-forecast-system-reanalysis-cfsr) or CFSR is a reanalysis data product giving a best estimate of the state of the coupled climate system since 1979 based on a blend of observations and numerical model.

> The CFSR is a third generation reanalysis product. It is a global, high resolution, coupled atmosphere-ocean-land surface-sea ice system designed to provide the best estimate of the state of these coupled domains over this period. The CFSR includes (1) coupling of atmosphere and ocean during the generation of the 6 hour guess field, (2) an interactive sea-ice model, and (3) assimilation of satellite radiances. The CFSR global atmosphere resolution is ~38 km (T382) with 64 levels. The global ocean is 0.25° at the equator, extending to a global 0.5° beyond the tropics, with 40 levels. The global land surface model has 4 soil levels and the global sea ice model has 3 levels. The CFSR atmospheric model contains observed variations in carbon dioxide (CO2), together with changes in aerosols and other trace gases and solar variations. With these variable parameters, the analyzed state will include estimates of changes in the Earth system climate due to these factors. The current CFSR will be extended as an operational, real time product into the future.

We maintain a continuously updated local copy of the 6-hourly CFSR data on shared disks here in DAES. You can browse the available data through this catalog:

http://thredds.atmos.albany.edu:8080/thredds/catalog/CFSR/catalog.html

That links takes you to a so-called THREDDS server which provides remote access to the data. However we're always going to get *much* better performance with local access to the file system.

In [1]:
thredds_path = 'http://thredds.atmos.albany.edu:8080/thredds/dodsC/CFSR/' 
local_path = '/cfsr/data/'

path = local_path  # switch this to thredds_path if running remotely ... but much poorer performance

Let's take a peak at one directory of data from a particular year:

In [2]:
ls /cfsr/data/2000/

g.2000.0p5.anl.nc        q.2000.0p5.anl.nc       u_pv.2000.0p5.anl.nc
pmsl.2000.0p5.anl.nc     t.2000.0p5.anl.nc       v.2000.0p5.anl.nc
pres_pv.2000.0p5.anl.nc  t_pv.2000.0p5.anl.nc    v_isen.2000.0p5.anl.nc
psfc.2000.0p5.anl.nc     tsfc.2000.0p5.anl.nc    v_pv.2000.0p5.anl.nc
pv_isen.2000.0p5.anl.nc  u.2000.0p5.anl.nc       w.2000.0p5.anl.nc
pwat.2000.0p5.anl.nc     u_isen.2000.0p5.anl.nc


We can see 17 different fields stored in individual netCDF files labeled with their calendar year.

Our goal is to **compute a 30-year seasonal climatology** for all variables. So we are going to begin by opening handles to 30 years of data.

In [3]:
# It's helpful to store all the variable names in this list
variables = ['g',
             'pmsl',
             'pres_pv',
             'psfc',
             'pv_isen',
             'pwat',
             'q',
             't',
             't_pv',
             'tsfc',
             'u',
             'u_isen',
             'u_pv',
             'v',
             'v_isen',
             'v_pv',
             'w',
            ]

## Opening handles to the dataset

Here we are going to assemble a list of paths to every data file that we want to concatenate into a single dataset.

### Assembling paths to all the data

In [4]:
pmsl_files = []
var = 'pmsl'
for y in range(1988,2018):  # choosing years 1988 through 2017 because there is some missing data after 2017
    #for var in variables:
    pmsl_filepath = path + str(y) + '/' + var + '.' + str(y) +'.0p5.anl.nc'
    pmsl_files.append(pmsl_filepath)

Let's take a peek at part of the list we just created:

In [5]:
pmsl_files

['/cfsr/data/1988/pmsl.1988.0p5.anl.nc',
 '/cfsr/data/1989/pmsl.1989.0p5.anl.nc',
 '/cfsr/data/1990/pmsl.1990.0p5.anl.nc',
 '/cfsr/data/1991/pmsl.1991.0p5.anl.nc',
 '/cfsr/data/1992/pmsl.1992.0p5.anl.nc',
 '/cfsr/data/1993/pmsl.1993.0p5.anl.nc',
 '/cfsr/data/1994/pmsl.1994.0p5.anl.nc',
 '/cfsr/data/1995/pmsl.1995.0p5.anl.nc',
 '/cfsr/data/1996/pmsl.1996.0p5.anl.nc',
 '/cfsr/data/1997/pmsl.1997.0p5.anl.nc',
 '/cfsr/data/1998/pmsl.1998.0p5.anl.nc',
 '/cfsr/data/1999/pmsl.1999.0p5.anl.nc',
 '/cfsr/data/2000/pmsl.2000.0p5.anl.nc',
 '/cfsr/data/2001/pmsl.2001.0p5.anl.nc',
 '/cfsr/data/2002/pmsl.2002.0p5.anl.nc',
 '/cfsr/data/2003/pmsl.2003.0p5.anl.nc',
 '/cfsr/data/2004/pmsl.2004.0p5.anl.nc',
 '/cfsr/data/2005/pmsl.2005.0p5.anl.nc',
 '/cfsr/data/2006/pmsl.2006.0p5.anl.nc',
 '/cfsr/data/2007/pmsl.2007.0p5.anl.nc',
 '/cfsr/data/2008/pmsl.2008.0p5.anl.nc',
 '/cfsr/data/2009/pmsl.2009.0p5.anl.nc',
 '/cfsr/data/2010/pmsl.2010.0p5.anl.nc',
 '/cfsr/data/2011/pmsl.2011.0p5.anl.nc',
 '/cfsr/data/201

Looks good!

### Passing through paths to `xarray.open_mfdataset()`

Now we pass our list of path and let Xarray (with Dask in the background) handle the concatenation:

In [6]:
import xarray as xr

pmsl_6hourly = xr.open_mfdataset(pmsl_files, parallel=True)
pmsl_6hourly

<xarray.Dataset>
Dimensions:  (time: 43832, lat: 361, lon: 720)
Coordinates:
  * time     (time) datetime64[ns] 1988-01-01 ... 2017-12-31T18:00:00
  * lat      (lat) float32 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables:
    pmsl     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
Attributes:
    description:    pmsl as a single level variable
    year:           1988
    source:         http://nomads.ncdc.noaa.gov/data.php?name=access#CFSR-data
    references:     Saha, et. al., (2010)
    created_by:     User: kgriffin
    creation_date:  Fri Apr 13 14:12:35 UTC 2012

We can see that we have a dataset with
- 1 data variable (`pmsl`)
- 1/4 degree lat-lon resolution
- 40 pressure levels
- 43,832 time elements (that's 4x daily for 30 years, accounting for leap years!)

And of course we can (and will) repeat this procedure for each of the variables in our list.

### Dask and lazy execution

Note that we've just loaded in handles to a LOT of data. Each 4-dimensional field (lat,lon,pressure,time) is roughly $4.6 \times 10^{11}$ data points, which at single-precision floating point storage works out to about 1.7 TB!

The total size of this 30-year dataset is about 22 TB.

How does this work? We definitely do not have 22 TB of available memory on this system!

This is an example of "lazy execution". We have not actually read all that data off of disk. All we have done is read the metadata to effectively create a map of the dataset so that Xarray knows which computations should be valid on this dataset.

## Creating the seasonal climatology

Now let's get started on *reducing* this dataset into a form more amenable to our analysis. We are going to take *time averages* over individual seasons.

### Grouping by season

Our time access already understands meteorological seasons DJF (December, January, February), JJA (June, July, August), etc. So creating groups for the four seasons is very simple. Here's a quick example of using a `groupby` operation:

In [7]:
pmsl_6hourly.groupby(pmsl_6hourly.time.dt.season)

DatasetGroupBy, grouped over 'season'
4 groups with labels 'DJF', 'JJA', 'MAM', 'SON'.

### Definining the climatology

We use a grouped operation to take averages over all data points within each season.

*Note, this operation is very fast because no actualy computations are being done at this point, just laying out a map for **how to do the computations**.*

In [8]:
pmsl_seas_clim = cfsr_6hourly.groupby(pmsl_6hourly.time.dt.season).mean()
pmsl_seas_clim

<xarray.Dataset>
Dimensions:  (lat: 361, lon: 720, season: 4)
Coordinates:
  * lat      (lat) float32 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    pmsl     (season, lat, lon) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>

Now we can see that the resulting dataset has a `seasons` axis with our four seasons 'DJF', 'MAM', 'JJA', 'SON'.

*With this time averaging, we have reduced the data from 22 TB down to about 2 GB.*

Our task is now to save the reduced dataset to disk, so that we can do our analyses on data that fits much more comfortably into memory.

*Now comes all the heavy lifting.* In order to save the climatology to disk, we have to actually carry out the computations. This part will be very slow.

## Saving the climatology to disk

The code is extremely simple to write, but the execution will be very slow.

In [9]:
pmsl_seas_clim.to_netcdf('/nfs/roselab_rit/data/cfsr_climatology/pmsl.seas_clim.0p5.nc')

## Doing it all again

In [12]:
import xarray as xr

pwat_files = []
var = 'pwat'
for y in range(1988,2018):  # choosing years 1988 through 2017 because there is some missing data after 2017
    #for var in variables:
    pwat_filepath = path + str(y) + '/' + var + '.' + str(y) +'.0p5.anl.nc'
    pwat_files.append(pwat_filepath)
pwat_6hourly = xr.open_mfdataset(pwat_files, parallel=True)
pwat_seas_clim = pwat_6hourly.groupby(pwat_6hourly.time.dt.season).mean()
pwat_seas_clim.to_netcdf('/nfs/roselab_rit/data/cfsr_climatology/pwat.seas_clim.0p5.nc')

In [ ]:
import xarray as xr

t_files = []
var = 't'
for y in range(1988,2018):  # choosing years 1988 through 2017 because there is some missing data after 2017
    #for var in variables:
    t_filepath = path + str(y) + '/' + var + '.' + str(y) +'.0p5.anl.nc'
    t_files.append(t_filepath)
t_6hourly = xr.open_mfdataset(t_files, parallel=True)
# Here we select just one pressure level
t1000_seas_clim = t_6hourly.sel(lev=1000.).groupby(t_6hourly.time.dt.season).mean()
t1000_seas_clim.to_netcdf('/nfs/roselab_rit/data/cfsr_climatology/t1000.seas_clim.0p5.nc')